In [1]:
from transformers import pipeline
# загрузка и предобработка
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import openpyxl, graphviz, sklearn, shap

# https://github.com/huggingface/transformers
# обучение, загрузка по формату библиотеки, оптимизаторы
# !pip install transformers
# !pip install datasets
# !pip install umap-learn
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
# обучение
import torch
from torch.utils.data import DataLoader
# предобработка и финальный отчет по метрикам
from sklearn import preprocessing
from sklearn.metrics import classification_report
# кластеризация
from sklearn.feature_extraction.text import CountVectorizer
import umap
from sklearn.cluster import DBSCAN
# прогресс бар
from tqdm import tqdm
# очистка кешей
import gc
# размер шрифта в графиках
import matplotlib
matplotlib.rcParams.update({'font.size': 22})

In [2]:
model_name = "model_f1_max=0.854261184506899"

tokenizer = AutoTokenizer.from_pretrained('ai-forever/ruBert-base')
# если есть желание дообучить лучшую модель - оставить эту строчку кода, иначе убрать
# model_name = 'sber-80(-84)'
model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                           num_labels=2, 
                                                           output_attentions=False,
                                                           output_hidden_states=False,
                                                           ignore_mismatched_sizes=True)

In [3]:
df_0 = pd.read_excel('Файл_для_применения_BERT.xlsx') #, sheet_name='Инфляция'

In [4]:
df_0.head(3)

,date,text,item,link,category,type
0,2024-12-01,Начатые властями Киргизии в отношении «Яндекс ...,В такси подсели проверки,https://www.kommersant.ru/doc/7344142,41,Инфляция и цены
1,2024-12-01,По итогам первых предварительных итогов «черно...,«Черная пятница» прошла в соединенно-штатном р...,https://www.kommersant.ru/doc/7344071,41,Инфляция и цены
2,2024-12-01,За год в России закрылось свыше 200 книжных ма...,В книжные магазины заглянул кризис,https://www.kommersant.ru/doc/7344021,41,Инфляция и цены


In [5]:
# df_1 = df_0[[df_0.type == 'Инфляция и цены']]

In [6]:
# df_1.head(3)

In [7]:
df = df_0['text']
type(df)

pandas.core.series.Series

In [8]:
df.to_csv('test.csv', index=False)

In [9]:
dataset = load_dataset('csv', data_files={'test': 'test.csv'})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 393
    })
})

In [11]:
dataset = dataset.map(lambda e: tokenizer(e['text'], truncation = True, max_length=300, padding='max_length'), batched=True)

Map:   0%|          | 0/393 [00:00<?, ? examples/s]

In [12]:
# это стандартные колонки для формата пайторча, все кроме них убираем
pytorch_style_columns = ['input_ids', 'token_type_ids', 'attention_mask']  #
# убираем их из загрузчика
dataset = dataset.remove_columns(list(set(list(dataset['test'].features.keys())) - set(pytorch_style_columns)))

In [13]:
dataset

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 393
    })
})

In [14]:
# переврдим в формат пайторча
# сразу грузим на gpu, если есть cuda, иначе девайс стоит убрать из аргумента
dataset.set_format(type='torch', columns=pytorch_style_columns, device='cuda')

In [15]:
# установить 8 или больше, если более 16 гб видеопамяти
test_dataloader = DataLoader(dataset['test'], shuffle=False, batch_size=1)

In [16]:
len(test_dataloader)

393

In [17]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [18]:
# np.save('classes.npy', Label_encoder.classes_)
Label_encoder = preprocessing.LabelEncoder()

# для того, чтобы восстановить кодировку целевых признаков как при обучении
Label_encoder.classes_ = np.load('classes.npy', allow_pickle=True)

# валидируем по тесту
# реальные ответы и предсказания
# true = []
preds = []
val = []

# переводим модель в режим инференса
model.eval()
# проходимся по батчам теста
for batch in tqdm(test_dataloader):
    # переносим батч на GPU
    batch = {k: v.to(device) for k, v in batch.items()}
    # без обновления параметров находим прогнозы
    with torch.no_grad():
        outputs = model(**batch)
    # получаем логиты
    logits = outputs.logits
    # из логитов прогноз
    predictions = torch.softmax(logits, dim=-1) #predictions = torch.argmax(logits, dim=-1)
#     print(round(float(predictions[0][1]),3))
    # добавляем пачки ответов и прогнозов в массивы
#     true += batch["labels"].detach().cpu().numpy().tolist()
#     preds += round(float(predictions[0][1],3))  #.detach().cpu().numpy().tolist()

    preds.append(float(predictions[0][1]))
    val.append(logits)
# ковертируем ответы и прогнозы обратно в человеко-читаемые названия классов, смотрим report по всем метрикам по каждому классу
# print(classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds)))

100%|████████████████████████████████████████████████████████████████████████████████| 393/393 [00:17<00:00, 23.06it/s]


In [19]:
preds

[0.010994292795658112,
 0.9693111777305603,
 0.010895106010138988,
 0.39859768748283386,
 0.8457617163658142,
 0.7684265375137329,
 0.009940050542354584,
 0.8032139539718628,
 0.8800523281097412,
 0.006371532566845417,
 0.9504613876342773,
 0.9669992327690125,
 0.9857250452041626,
 0.6915534138679504,
 0.02598835900425911,
 0.6450362801551819,
 0.9839782118797302,
 0.8389288187026978,
 0.15694794058799744,
 0.9530960321426392,
 0.9881312847137451,
 0.9425320625305176,
 0.8956959247589111,
 0.9754029512405396,
 0.00585210183635354,
 0.06489956378936768,
 0.8276351094245911,
 0.1074289008975029,
 0.9578237533569336,
 0.9759758114814758,
 0.9844833016395569,
 0.3292654752731323,
 0.9826241135597229,
 0.43730854988098145,
 0.08841470628976822,
 0.9196351170539856,
 0.016010446473956108,
 0.006923337001353502,
 0.9857336282730103,
 0.9870261549949646,
 0.010615035891532898,
 0.008796686306595802,
 0.9920353293418884,
 0.886360228061676,
 0.9859922528266907,
 0.9371753931045532,
 0.953404724

In [33]:
label = []
for prob in preds:
    if prob < 0.8:
        label.append('Проинфляционный')
    elif prob > 0.8 and prob < 0.9:
        label.append('Нейтральный')
    else:
        label.append('Дезинфляционный')
pd.DataFrame(label).value_counts()

Проинфляционный    207
Дезинфляционный    155
Нейтральный         31
dtype: int64

In [34]:
df_0.head(3)

,date,text,item,link,category,type,label
0,2024-12-01,Начатые властями Киргизии в отношении «Яндекс ...,В такси подсели проверки,https://www.kommersant.ru/doc/7344142,41,Инфляция и цены,Проинфляционный
1,2024-12-01,По итогам первых предварительных итогов «черно...,«Черная пятница» прошла в соединенно-штатном р...,https://www.kommersant.ru/doc/7344071,41,Инфляция и цены,Дезинфляционный
2,2024-12-01,За год в России закрылось свыше 200 книжных ма...,В книжные магазины заглянул кризис,https://www.kommersant.ru/doc/7344021,41,Инфляция и цены,Проинфляционный


In [35]:
# df_0.drop('Unnamed: 0', axis=1, inplace=True)

In [36]:
df_0['label'] = label

In [37]:
df_0

,date,text,item,link,category,type,label
0,2024-12-01,Начатые властями Киргизии в отношении «Яндекс ...,В такси подсели проверки,https://www.kommersant.ru/doc/7344142,41,Инфляция и цены,Проинфляционный
1,2024-12-01,По итогам первых предварительных итогов «черно...,«Черная пятница» прошла в соединенно-штатном р...,https://www.kommersant.ru/doc/7344071,41,Инфляция и цены,Дезинфляционный
2,2024-12-01,За год в России закрылось свыше 200 книжных ма...,В книжные магазины заглянул кризис,https://www.kommersant.ru/doc/7344021,41,Инфляция и цены,Проинфляционный
3,2024-12-02,"Дискуссия Центробанка, правительства и крупног...",ЦБ много не берет,https://www.kommersant.ru/doc/7309663,3,Инфляция и цены,Проинфляционный
4,2024-12-02,«Сбер» поднял ставки по рыночным кредитам на ж...,«Сбер» последовал за ключевой ставкой,https://www.kommersant.ru/doc/7309599,3,Инфляция и цены,Нейтральный
...,...,...,...,...,...,...,...
388,2025-01-16,Котировки золота на мировом рынке после месячн...,Инаурумация Дональда Трампа,https://www.kommersant.ru/doc/7433478,40,Инфляция и цены,Дезинфляционный
389,2025-01-16,Индекс Московской биржи (MOEX: MOEX) в начале ...,Индекс Мосбиржи резко вырос на открытии торгов,https://www.kommersant.ru/doc/7433014,40,Инфляция и цены,Дезинфляционный
390,2025-01-16,В отличие от других сегментов корпоративного ф...,Факторы отыграли срочность,https://www.kommersant.ru/doc/7432786,40,Инфляция и цены,Дезинфляционный
391,2025-01-17,Сегмент автолизинга в 2024 году покажет отрица...,Автолизинг покатился,https://www.kommersant.ru/doc/7433524,4,Инфляция и цены,Проинфляционный


In [38]:
# df_0.to_excel('Результат/Размеченные_новости_2024-12-02.xlsx')
df_0.to_excel('Результат/Размеченные_новости_новый_порог_2025-17-01.xlsx')

In [39]:
final = df_0['label'].groupby([df_0['date'].dt.year, df_0['date'].dt.month]).value_counts()

In [40]:
final

date  date  label          
2024  12    Проинфляционный    163
            Дезинфляционный    129
            Нейтральный         25
2025  1     Проинфляционный     44
            Дезинфляционный     26
            Нейтральный          6
Name: label, dtype: int64

In [41]:
final.to_excel('Результат/Оцифрованные_новости_новый_порог_2025-17-01.xlsx')

In [42]:
count_of_news = df_0.label.groupby([df_0['date'].dt.year, df_0['date'].dt.month]).count()

In [43]:
count_of_news.to_excel('Количество дек-янв.xlsx')